### Airline Review - Data Cleaning

In this notebook, the following steps are taken: 

1. Load original data
2. Delete blank rows
3. Impute Null Values
4. Assign nationality of airline
5. Calculate length of review, time between flight and review
6. Save to .csv

In [93]:
# Import Libraries
import pandas as pd
import numpy as np
import math
import re
import datetime

In [94]:
# Load original dataset
airline = pd.read_excel('/Users/paulhershaw/brainstation_course/airplane_project/data/airline_reviews.xlsx')

In [95]:
# Review the original dataset
airline.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
airline,65947,81,Spirit Airlines,2934,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overall,64017.0,NaN,NaN,NaN,5.14543,3.477532,1.0,1.0,5.0,9.0,10.0
author,65947,44069,Anders Pedersen,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_date,65947,3015,19th January 2015,253,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_review,65947,61172,On March 2/14 a friend and I were booked on an...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aircraft,19718,2088,A320,2157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
traveller_type,39755,4,Solo Leisure,14798,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cabin,63303,4,Economy Class,48558,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route,39726,24549,Bangkok to Hong Kong,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_flown,39633,63,August 2015,1204,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# Drop blank rows.
airline.dropna(axis = 0, how = 'all', inplace = True)


# Remove "Trip Verified"
airline['customer_review'] = airline.loc[:,'customer_review'].map(lambda exp:exp.split('| ')[1] if "Trip Verified" in exp else exp)
# Remove "not Verified"
airline['customer_review'] = airline.loc[:,'customer_review'].map(lambda exp:exp.split('|')[1] if 'not verified' in exp else exp)
# Remove"Verified review"
airline['customer_review'] = airline.loc[:,'customer_review'].map(lambda exp:exp.split('|')[1] if 'verified review' in exp else exp)


# Convert recommended columns to binary (yes=1,No=0). This will be our target variable.
airline['recommended'] = airline['recommended'].map({'yes':1,'no':0})
airline['recommended'] = airline.apply(lambda row: 1 if row['overall'] > 5 else 0, axis=1)


# Drop all rows with null values in the overall column
airline.dropna(subset=['overall'], inplace=True)

#Impute values for rankings. 
columns_to_update = ['seat_comfort', 'cabin_service', 'food_bev', 'entertainment', 'ground_service', 'value_for_money']
for column in columns_to_update:
    airline[column].fillna(airline['overall'] / 2, inplace=True)

# Filter and impute null values using mode with transform
airline['cabin'] = airline.groupby('airline')['cabin'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))

# Convert date_flown to datetime
airline['date_flown'] = pd.to_datetime(airline['date_flown'], errors='coerce')
airline['review_date_updated1'] = pd.to_datetime(airline['review_date'], errors='coerce')


In [98]:
# Function to censor the review date
def safe_clean_and_convert_date(date_str):
    try:
        # Use regular expression to remove ordinal indicators specifically after day numbers
        cleaned_date = re.sub(r'(?<=\d)(st|nd|rd|th)', '', date_str)
        
        # Convert to datetime object
        return datetime.datetime.strptime(cleaned_date, '%d %B %Y')
    except ValueError:
        # Return None if the date cannot be parsed
        return None

In [99]:
# Create a new column and apply the date cleaning function
airline['review_date_updated2'] = None
airline.loc[airline['review_date_updated1'].isna(), 'review_date_updated2'] = airline.loc[airline['review_date_updated1'].isna(), 'review_date'].apply(safe_clean_and_convert_date)
airline['review_date_final'] = airline['review_date_updated1'].fillna(airline['review_date_updated2'])
airline.drop(['review_date','review_date_updated1', 'review_date_updated2'], axis=1, inplace=True)

#Calculate the days between the review and the flight
airline.loc[:, 'days_between_flight_and_review'] = (airline['review_date_final'] - airline['date_flown']).dt.days

In [100]:
# Dictionary of airline countries
airline_countries = {
    'Turkish Airlines': 'Turkey',
    'Qatar Airways': 'Qatar',
    'Emirates': 'United Arab Emirates',
    'Lufthansa': 'Germany',
    'KLM Royal Dutch Airlines': 'Netherlands',
    'Virgin America': 'United States',
    'American Airlines': 'United States',
    'Delta Air Lines': 'United States',
    'Southwest Airlines': 'United States',
    'United Airlines': 'United States',
    'Jetblue Airways': 'United States',
    'Aegean Airlines': 'Greece',
    'Aeroflot Russian Airlines': 'Russia',
    'Aeromexico': 'Mexico',
    'Air Canada': 'Canada',
    'Air New Zealand': 'New Zealand',
    'Alitalia': 'Italy',
    'AirAsia': 'Malaysia',
    'Asiana Airlines': 'South Korea',
    'Avianca': 'Colombia',
    'Austrian Airlines': 'Austria',
    'British Airways': 'United Kingdom',
    'Brussels Airlines': 'Belgium',
    'China Eastern Airlines': 'China',
    'China Southern Airlines': 'China',
    'Copa Airlines': 'Panama',
    'Ethiopian Airlines': 'Ethiopia',
    'Egyptair': 'Egypt',
    'Finnair': 'Finland',
    'Iberia': 'Spain',
    'ANA All Nippon Airways': 'Japan',
    'easyJet': 'United Kingdom',
    'Korean Air': 'South Korea',
    'LATAM Airlines': 'Chile',
    'LOT Polish Airlines': 'Poland',
    'Qantas Airways': 'Australia',
    'Air France': 'France',
    'Etihad Airways': 'United Arab Emirates',
    'Pegasus Airlines': 'Turkey',
    'Royal Jordanian Airlines': 'Jordan',
    'Ryanair': 'Ireland',
    'South African Airways': 'South Africa',
    'Saudi Arabian Airlines': 'Saudi Arabia',
    'TAP Portugal': 'Portugal',
    'Eurowings': 'Germany',
    'EVA Air': 'Taiwan',
    'Royal Air Maroc': 'Morocco',
    'Turkish Airlines': 'Turkey',
    'Qatar Airways': 'Qatar',
    'Emirates': 'United Arab Emirates',
    'Lufthansa': 'Germany',
    'KLM Royal Dutch Airlines': 'Netherlands',
    'Virgin America': 'United States',
    'American Airlines': 'United States',
    'Delta Air Lines': 'United States',
    'Southwest Airlines': 'United States',
    'United Airlines': 'United States',
    'Jetblue Airways': 'United States',
    'Aegean Airlines': 'Greece',
    'Aeroflot Russian Airlines': 'Russia',
    'Aeromexico': 'Mexico',
    'Air Canada': 'Canada',
    'Air New Zealand': 'New Zealand',
    'Alitalia': 'Italy',
    'AirAsia': 'Malaysia',
    'Asiana Airlines': 'South Korea',
    'Avianca': 'Colombia',
    'Austrian Airlines': 'Austria',
    'British Airways': 'United Kingdom',
    'Brussels Airlines': 'Belgium',
    'China Eastern Airlines': 'China',
    'China Southern Airlines': 'China',
    'Copa Airlines': 'Panama',
    'Ethiopian Airlines': 'Ethiopia',
    'Egyptair': 'Egypt',
    'Finnair': 'Finland',
    'Iberia': 'Spain',
    'ANA All Nippon Airways': 'Japan',
    'easyJet': 'United Kingdom',
    'Korean Air': 'South Korea',
    'LATAM Airlines': 'Chile',
    'LOT Polish Airlines': 'Poland',
    'Qantas Airways': 'Australia',
    'Air France': 'France',
    'Etihad Airways': 'United Arab Emirates',
    'Pegasus Airlines': 'Turkey',
    'Royal Jordanian Airlines': 'Jordan',    'Singapore Airlines': 'Singapore',
    'SAS Scandinavian': 'Multinational (Denmark, Norway, Sweden)',
    'Swiss Intl Air Lines': 'Switzerland',
    'Thai Airways': 'Thailand',
    'Air India': 'India',
    'Air Europa': 'Spain',
    'Air Canada rouge': 'Canada',
    'airBaltic': 'Latvia',
    'Air China': 'China',
    'Cathay Pacific Airways': 'Hong Kong',
    'Wizz Air': 'Hungary',
    'Spirit Airlines': 'United States',
    'TAROM Romanian': 'Romania',
    'Vueling Airlines': 'Spain',
    'Sunwing Airlines': 'Canada',
    'QantasLink': 'Australia',
    'Bangkok Airways': 'Thailand',
    'flydubai': 'United Arab Emirates',
    'Garuda Indonesia': 'Indonesia',
    'Germanwings': 'Germany',
    'Frontier Airlines': 'United States',
    'Icelandair': 'Iceland',
    'IndiGo': 'India',
    'Aer Lingus': 'Ireland',
    'Adria Airways': 'Slovenia',
    'Air Arabia': 'United Arab Emirates',
    'Alaska Airlines': 'United States',
    'Tunisair': 'Tunisia',
    'Norwegian': 'Norway',
    'Thai Smile Airways': 'Thailand',
    'Gulf Air': 'Bahrain',
    'Kuwait Airways': 'Kuwait',
    'WOW air': 'Iceland',
    'Ukraine International': 'Ukraine'
}

# Create a new column with the home country of the airline
airline['home_country'] = airline['airline'].map(airline_countries)

In [101]:
# Calculate the word count of the review
airline['review_word_count'] = airline['customer_review'].str.split().str.len()

In [102]:
# Drop duplicates from 'custoer_review' and reset index
airline.drop_duplicates(subset=['customer_review'],inplace=True)
airline.reset_index(level=0, inplace=True,drop=True)

In [103]:
# Shift columns
airline = airline[['airline', 
                   'home_country', 
                   'customer_review', 
                   'review_word_count', 
                   'cabin', 
                   'date_flown', 
                   'days_between_flight_and_review', 
                   'overall', 
                   'recommended',
                   'seat_comfort', 
                   'cabin_service',
                   'food_bev', 
                   'entertainment', 
                   'ground_service', 
                   'value_for_money']]

In [104]:
#Save to data folder
airline.to_csv('/Users/paulhershaw/brainstation_course/airplane_project/data/airline_reviews_cleaned.csv', index=False)